# ARCE experiments-03: adding $H_\text{eff}$

The most straigtforward setting, where both $h$ and $P(h)$ are observable.

Add bias via putting some spurious correlation in the prompt (and change the name of the objects).
Like "Turn off the screen after experiment". And change the name of the object at the same time.

## 0. Prepare $h$ and $d^0$

In [1]:
import toml
import numpy as np
import torch
import random
import pandas as pd
import json
import time
from tqdm import tqdm
from matplotlib import pyplot as plt
from itertools import permutations
import os
import openai
import copy

from utils.gpt_api import multi_turn_chatgpt
from utils.text_logger import text_logger
from utils.h_and_d import data_generator
from utils.h_and_d import h_x, cnt_of_status, gen_hstar_rnd, gen_hstar_given_status
from utils.standard_prompts import gen_hd_prompt, gen_dh_prompt, gen_data_prompt, gen_h_refine_prompt, gen_h_search_prompt
# from utils.evaluations import eval_feedback_h, convert_chatcompl_to_json
from utils.evaluations import *

def rnd_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

H_INDEX = 132 #[6, 26, 27, 33, 77, 78, 83, 111, 128, 132]
GLOBAL_TMP = 1 # [1, 0.5, 0.1]
SEED = 42 # [10086, 14843, 42, 1314, 916, 1024]
GEN = 4
M_generate = 4   # How many example generated by agent for the next generation
M_d0 = 8
LOOK_BACK = 2

INTERACTION = 'self'  # [self, hypothesis, none]
BIAS_START = 'strong' # [strong, medium, weak, none]
BIAS_END = 'none' # [strong, medium, weak, none]
BIAS_NAME = 'start%s_end%s'%(BIAS_START, BIAS_END)
MODEL_NAME =   "mistral-small-2312"  # ["gpt-3.5-turbo-0125", "gpt-4-0125-preview", "claude-3-haiku-20240307", "mistral-tiny-2312", "mistral-small-2312"]

rnd_seed(SEED)

hindex = "h"+str(H_INDEX).zfill(3)
EXP_NAME = "%s_%s_%s_seed%d"%(INTERACTION, hindex, BIAS_NAME, SEED)
EXP_PATH = './exp_logs_' + MODEL_NAME + '/add_interaction'
exp_path = os.path.join(EXP_PATH, EXP_NAME)
LOG = text_logger(file_name='chat_log', exp_path=exp_path, silence = True)
LOG.write_to_file('This is an experiment trying to see the convergence of H(P(h))')
# ------- Exp_name follows the section of h

In [2]:
OBJECTS = ['A', 'B', 'C', 'D', 'screen']
STATES = ["on","off","und"]
N = len(OBJECTS)
# ------------- Generate h satisfying 
def generate_all_statuses(N,M, packed=False):
    def generate_all_statuses_(N, M):
        def generate_status_helper(current_status):
            if len(current_status) == N:
                all_statuses.append(current_status.copy())
                return
            for state in range(M):
                current_status.append(STATES[state])
                generate_status_helper(current_status)
                current_status.pop()
        all_statuses = []
        generate_status_helper([])
        return all_statuses
    tmp_all_possible_statuses = generate_all_statuses_(N, M)
    if packed:
        all_possible_statuses = []
        for s in tmp_all_possible_statuses:
            all_possible_statuses.append(s[::-1])
        return all_possible_statuses
    else:
        return tmp_all_possible_statuses
    
all_possible_statuses = generate_all_statuses(N=len(OBJECTS), M=len(STATES), packed=False)

hcomp_df = pd.DataFrame(columns=['A','B','C','D','screen','status','n_on','n_off','n_und'])
feasible_status = []
index = 0
for l in all_possible_statuses:
    if l[-1]=='off':
        pass
    else:
        feasible_status.append(l)
        n_on, n_off, n_und = cnt_of_status(l)
        hcomp_df.loc[index] = [l[0],l[1],l[2],l[3],l[4], [l[0],l[1],l[2],l[3],l[4]], n_on, n_off, n_und]
        index += 1
        
#hcomp_df.sort_values(by=["n_on"],ascending=False)

In [3]:
h_star = gen_hstar_given_status(hcomp_df.iloc[[H_INDEX]]['status'][H_INDEX], OBJECTS)
h_bar = copy.deepcopy(h_star)
h_bar['screen'] = 'off'

print(h_star)
print(h_bar)
# --------- Get d0 given h_star
D_GENERATOR = data_generator(h_star, N_test=0)
d0 = D_GENERATOR.sample_d0_under_hbar(M=M_d0, h_bar=h_bar)

LOG.write_to_file('In this experiment,h* is %s, h_bar is %s. Global temperature is %f\n'%(h_star, h_bar, GLOBAL_TMP))

{'A': 'und', 'B': 'off', 'C': 'off', 'D': 'on', 'screen': 'on'}
{'A': 'und', 'B': 'off', 'C': 'off', 'D': 'on', 'screen': 'off'}


In [4]:
#d0

## 1. Run experiments with modified prompt and name

In [5]:
if BIAS_START == 'strong':
    spu_corr1 = "Turn off the screen after the experiment."
elif BIAS_START == 'medium':
    spu_corr1 = "John will turn off the screen after the experiment."
elif BIAS_START == 'weak':
    spu_corr1 = "John will close the screen after the experiment."
else:
    spu_corr1 = ""

if BIAS_END == 'strong':
    spu_corr2 = "\nTurn off the screen after the experiment."
elif BIAS_END == 'medium':
    spu_corr2 = "\nJohn will turn off the screen after the experiment."
elif BIAS_END == 'weak':
    spu_corr2 = "\nJohn will close the screen after the experiment."
else:
    spu_corr2 = ""

# ========== In this game_prompt, we add one scentence before d0
# spu_corr1 = "Turn off the screen after the experiment."
# #"John will turn off the screen after the experiment."#"Turn off the screen of the monitor."#"Turn off the screen after the experiment."

game_prompt = 'Let us play a rule following game. You need to generate a rule that maps the following inputs to their corresponding outputs, or generate examples following a given rule. Each example is an input-output pair. The input is a list of objects. If any objects with status on in the input, the output should be on. If all objects in the input are off, the output is off. If only objects with off and und in the list, the output should be undetermined (und for short). The output is either "on", "off", or "und", indicating the state of the light. For each object, determine whether it triggers the light to turn on, does not trigger it, or if it is undetermined. We only have %d objects in this game: %s. %s\n'%(N, OBJECTS, spu_corr1)
LOG.msg_to_gpt(game_prompt)
GPT_AGENT = multi_turn_chatgpt(model=MODEL_NAME, temperature=GLOBAL_TMP, top_p=1, logger=LOG, game_description=game_prompt)

# ------------ Step 1: random choose M training data samples as d0
# spu_corr2 = ""#"\nTurn off the screen after the experiment."#"\nClose the screen after experiment."#"\nTurn off the screen after the experiment."#
if MODEL_NAME.startswith('claude'):
    rule_format = 'Rule: {"object 1": <"on"/"off"/"und">, "object 2": <"on"/"off"/"und">, ...}%s\nWe only have five objects, they are ["A", "B", "C", "D", "screen"].'%spu_corr2
elif MODEL_NAME.startswith('mistral'):
    rule_format = 'Rule: {"object 1": <"on"/"off"/"und">, "object 2": <"on"/"off"/"und">, ...}%s.'%spu_corr2
else:
    rule_format = 'Rule: {"object 1": <"on"/"off"/"und">, "object 2": <"on"/"off"/"und">, ...}%s'%spu_corr2
#rule_format = 'Rule: {object 1 on/off/und; object 2 on/off/und; ...}'

data_str = gen_data_prompt(d0, need_stat=True)
d0_str = data_str
results = {'nh_corr':[],'nh_perf':[],'d_sampled':[],'prompt_token':[],'rules':[],'rules_refine':[]}
results['d_sampled'].append(data_str)
results['rules'].append(h_star)

results_prob_list = []
for g in tqdm(range(GEN)):
    LOG.write_to_file('----------- Gen %d -----------'%g)
    # ------------ Step 2: h~P(h|d)
    hd_prompt = gen_hd_prompt(data=data_str, ask_rule=rule_format)
        # --------- Get feedback
    hd_feedback, hd_fb_probs, cnt_tokens = GPT_AGENT.call_chatgpt(hd_prompt, fake_response=None, 
                                              logprobs=True, top_logprobs=5, lookback_round=LOOK_BACK)
    results['rules'].append(hd_feedback)
    results_prob_list.append(hd_fb_probs)
    results['prompt_token'].append(cnt_tokens)

    try:
        hd_feedback_str = json.loads(hd_feedback.split("Rule: ")[-1])
    except:
        hd_feedback_str = json.loads(hd_feedback.split("Rule: ")[-1].split('\n')[0])
    
    nh_corr, nh_perf = eval_feedback_h(h=h_star, fb_h=hd_feedback_str, N=N)
    results['nh_corr'].append(nh_corr/N)
    results['nh_perf'].append(nh_perf)
    
        # ------------ Step 2.5: refine the hypothesis
    
    if INTERACTION.lower() == "self":
        h_refine_prompt = gen_h_search_prompt(hd_feedback_str, d0_str, rule_format)
        gpt_feedback, _ , cnt_tokens = GPT_AGENT.call_chatgpt(h_refine_prompt, lookback_round=LOOK_BACK, update_hist=False)
        tmp = json.loads(gpt_feedback)
        if 'Rule' in tmp.keys():
            hd_refi_fb_str = tmp['Rule']
        else:
            hd_refi_fb_str = tmp['rule']
        results['rules_refine'].append(hd_refi_fb_str)
    elif INTERACTION.lower() == "hypothesis":
        h_refine_prompt, refine_flag = gen_h_refine_prompt(hd_feedback_str, d0_str, rule_format)
        hrefine_feedback, _ , cnt_tokens = GPT_AGENT.call_chatgpt(h_refine_prompt, lookback_round=LOOK_BACK, update_hist=False)
        if refine_flag:     # --- h is not perfect correct, trigger refine
            hd_refi_fb_str = hrefine_feedback.split('Rule: ')[1]
            results['rules_refine'].append(hd_refi_fb_str)
        else:
            hd_refi_fb_str = hd_feedback_str
            results['rules_refine'].append(hd_feedback_str) 
    else:
        hd_refi_fb_str = hd_feedback_str
        results['rules_refine'].append(hd_feedback_str) 
    
    results['prompt_token'].append(cnt_tokens)
    if g<GEN-1:
        # ------------ Step 3: d~P(d|h)
        dh_prompt = gen_dh_prompt(M = M_generate, rule = hd_refi_fb_str)
            # --------- Get feedback
        dh_feedback, _, cnt_tokens = GPT_AGENT.call_chatgpt(dh_prompt, fake_response=None, lookback_round=LOOK_BACK)
        results['prompt_token'].append(cnt_tokens)
        results['d_sampled'].append(dh_feedback)
        # ----------- Step 4: data_str <-- dh_feedback
        data_str = dh_feedback
    #time.sleep(30)
print(results['nh_corr'])
print(results['prompt_token'])

  0%|          | 0/4 [00:02<?, ?it/s]


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# ------------ Save prob_lists
if MODEL_NAME.startswith('gpt'):
    file_name = 'prob_list_all.json'
    save_path = os.path.join(exp_path, file_name)    
    json.dump(convert_chatcompl_to_json(results_prob_list), open(save_path, 'w' ))    

file_name2 = 'other_results_all.json'
save_path2 = os.path.join(exp_path, file_name2)
json.dump(results, open(save_path2, 'w' ))

In [ ]:
# print('======= Before refine ========')
# for i in range(GEN+1):
#     if i==0:
#         print('Rul*: ',end='')
#     print(results['rules'][i])
    
# print('======= After refine ========')
# print('*%s'%results['rules'][0])
# for i in range(GEN):
#     print(' %s'%results['rules_refine'][i])

## Calculate necessary quantities

Given other_results_all.json, calculate:

1. Number of correct prediction in $d^0$ for each gen
2. Number of screen:off for each gen
3. Whether $h^{T*}=h^t$ for each gen

In [ ]:
def dstr_to_pairs(d_str):
    tmp_str = d_str.split('\n')
    while "" in tmp_str:
        tmp_str.remove("")
    data_pairs = []
    for s in range(int(len(tmp_str)*0.5)):
        tmp_input = tmp_str[2*s].split(': ')[1].split(', ')
        tmp_output = tmp_str[2*s+1].split(': ')[1]
        data_pairs.append((tmp_input, tmp_output))
    return data_pairs

def count_corr_d0_pairs(d_pairs, rule):
    corr_cnt, all_cnt = 0, 0
    for x,y in d_pairs:
        all_cnt += 1
        if h_x(x, rule)==y:
            corr_cnt += 1
    return corr_cnt, all_cnt

def eval_get_corr_h(results_read):
    h_corr_list = []
    h_star_read = results_read['rules'][0]
    for i in range(GEN-1):
        dt_str = results_read['d_sampled'][i+1].split('\n')
        while "" in dt_str:
            dt_str.remove("")
        dt_pairs = dlist_to_pairs(dt_str)
        corr_cnt, _ = count_corr_d0_pairs(dt_pairs, h_star_read)
        h_corr_list.append(corr_cnt)
    return h_corr_list

def cal_screen_off_hbar(rule_list, h_bar):
    cnt_screen, cnt_h = 0, 0
    for r in rule_list:
        if r['screen']=='off':
            cnt_screen += 1
        if r==h_bar:
            cnt_h += 1
    return cnt_screen, cnt_h, len(rule_list)

def updata_stats(stats, results_read,):
    d0_str = results_read['d_sampled'][0]   # Get d0 from the log
    d0_corr_list = eval_get_corr_d0(d0_str, results_read)  # How many d0 each ht can correctly predict
    h_bar = results_read['rules'][0]
    h_bar['screen']='off'
    cnt_screen_start, cnt_h, all_cnt = cal_screen_off_hbar(results_read['rules_refine'][:0], h_bar)
    cnt_screen, cnt_h, all_cnt = cal_screen_off_hbar(results_read['rules_refine'][-AVG_BACK:], h_bar)
    cnt_corr_d0 = np.sum(d0_corr_list[-AVG_BACK:])
    all_cnt_corr_d0 = AVG_BACK*8
    stats["cnt_screen_start"].append(cnt_screen_start)
    stats["cnt_screen_end"].append(cnt_screen)
    stats["cnt_h"].append(cnt_h)
    stats["cnt_all"].append(all_cnt)
    stats["cnt_d0"].append(cnt_corr_d0)
    stats["cnt_d0_all"].append(all_cnt_corr_d0)

def regularize_results(results_read):    
    for i in range(len(results_read['rules'])):
        if type(results_read['rules'][i]) != dict:
            results_read['rules'][i] = json.loads(results_read['rules'][i].split("Rule: ")[1])

    for i in range(len(results_read['rules_refine'])):
        if type(results_read['rules_refine'][i]) != dict:
            results_read['rules_refine'][i] = json.loads(results_read['rules_refine'][i])  
    return results_read

def get_results_read(exp_path_load):
    #save_path = os.path.join(exp_path_load, 'prob_list_all.json') 
    save_path2 = os.path.join(exp_path_load, 'other_results_all.json')
    #prob_list_read = json.load( open( save_path ))
    results_read = json.load(open(save_path2))
    return results_read

In [ ]:
# exp_path = "E://P5_iICL//iterated_learning_exp//exp_logs_gpt-3.5-turbo-0125//add_interaction//hypothesis_h033_startnone_endnone_seed42"
if MODEL_NAME.startswith('gpt'):
    save_path = os.path.join(exp_path, 'prob_list_all.json')
    prob_list_read = json.load( open( save_path ))
    
save_path2 = os.path.join(exp_path, 'other_results_all.json')
results_read = json.load(open(save_path2))
results_read = regularize_results(results_read)
GEN = len(results_read['nh_corr'])

In [ ]:
# ----------- How many correct d0 each ht can predict
d0_str = results_read['d_sampled'][0]   # Get d0 from the log
d0_corr_list = eval_get_corr_d0(d0_str, results_read)  # How many d0 each ht can correctly predict

# ----------- How many screen:off in each ht
ht_screenoff_list = []

# ----------- How many ht == h*
h_tgt = results_read['rules'][0]
h_tgt['screen'] = 'off'
ht_tgt_list = []

for i in range(len(results_read['rules_refine'])):
    tmp_screenoff = int(results_read['rules_refine'][i]['screen']=='off')
    ht_screenoff_list.append(tmp_screenoff)
    #ht_tgt_list.append(int(results_read['rules_refine'][i]==h_tgt))
    ht_tgt_list.append(int(tmp_screenoff==1 and d0_corr_list[i]==8))

print("----------- How many correct d0 each ht can predict")
print(d0_corr_list)
print("----------- How many screen:off in each ht")
print(ht_screenoff_list)
# print("----------- How many ht == h*")
print(ht_tgt_list)

In [ ]:
print(a)

## Visualize results (backup) don't delete, the reporting code is good

In [ ]:
# ============= Assign number of correct predictions to each h


def dlist_to_pairs(d_list):
    data_pairs = []
    if not (d_list[0].startswith('Input') or d_list[0].startswith('Output')):
        d_list = d_list[1:]
    for i in range(int(0.5*len(d_list))):
        tmp_input = d_list[2*i].split(': ')[1].split(', ')
        tmp_output = d_list[2*i+1].split(': ')[1]
        data_pairs.append((tmp_input, tmp_output))
    return data_pairs

def count_corr_d0_pairs(d_pairs, rule):
    corr_cnt, all_cnt = 0, 0
    for x,y in d_pairs:
        all_cnt += 1
        if h_x(x, rule)==y:
            corr_cnt += 1
    return corr_cnt, all_cnt

def count_corr_d0(d, h):
    # Calculate how many examples in d can be explained by given h
    corr_cnt = 0
    for _, row in d.iterrows():
        x, y = row['obj'], row['stat']
        y_pred = h_x(x, h)
        if y==y_pred:
            corr_cnt += 1
    return corr_cnt

def cal_entropy(hd_fb_probs):
    token_logprob = extract_probs(hd_fb_probs, OBJECTS, top_n=5)
    entropy = 0
    for i in range(len(all_possible_statuses)):
        h_tmp = {}
        for j in range(len(OBJECTS)):
            h_tmp[OBJECTS[j]] = all_possible_statuses[i][j]
        if h_tmp == h_star:
            h_star_idx = i
        obj_logprob, obj_prob = cal_prob_of_h(h_tmp, token_logprob)
        entropy += -obj_prob*obj_logprob
    return entropy

# ============= Assign number of correct predictions to each h
def count_corr_d0(d, h):
    # Calculate how many examples in d can be explained by given h
    corr_cnt = 0
    for _, row in d.iterrows():
        x, y = row['obj'], row['stat']
        y_pred = h_x(x, h)
        if y==y_pred:
            corr_cnt += 1
    return corr_cnt

def cal_entropy(hd_fb_probs):
    token_logprob = extract_probs(hd_fb_probs, OBJECTS, top_n=5)
    entropy = 0
    for i in range(len(all_possible_statuses)):
        h_tmp = {}
        for j in range(len(OBJECTS)):
            h_tmp[OBJECTS[j]] = all_possible_statuses[i][j]
        if h_tmp == h_star:
            h_star_idx = i
        obj_logprob, obj_prob = cal_prob_of_h(h_tmp, token_logprob)
        entropy += -obj_prob*obj_logprob
    return entropy

def draw_pic(hd_fb_probs, ax=None, y_log=True, ylim=None):
    if ax is None:
        fig, ax = plt.subplots(1,1,figsize=(15,5))
    token_logprob = extract_probs(hd_fb_probs, OBJECTS, top_n=5)

    prob_list, corr_list = [], []
    for i in range(len(all_possible_statuses)):
        h_tmp = {}
        for j in range(len(OBJECTS)):
            h_tmp[OBJECTS[j]] = all_possible_statuses[i][j]
        if h_tmp == h_star:
            h_star_idx = i
        obj_logprob, obj_prob = cal_prob_of_h(h_tmp, token_logprob)
        prob_list.append(obj_prob)
        corr_cnt = count_corr_d0(d0, h_tmp)
        corr_list.append(corr_cnt)
    prob_list = np.array(prob_list)
    corr_list = np.array(corr_list)

    ALPHA_LIST = [0.05, 0.2, 0.3, 0.5, 1]
    x_axis = np.arange(0,243)

    for i in range(N):
        mask = corr_list==i
        ax.bar(x_axis[mask],prob_list[mask],width=1, color='royalblue',alpha=ALPHA_LIST[i],label='h(d0)='+str(i))
    if y_log:
        ax.set_yscale('log')
    ax.set_xlim(-3, 245)
    if ylim is not None:
        ax.set_ylim(ylim)
    ax.plot((h_star_idx), (prob_list[h_star_idx]), color='red',alpha=1,linestyle=' ',marker='*',markersize=10, label='h*')
    ax.legend(fontsize=16, ncol=2)

def get_all_prob_lists(prob_list_read):
    off_prob_list = []
    on_prob_list = []
    und_prob_list = []
    for g in range(GEN):
        off_flg, on_flg, und_flg = False, False, False
        for i in range(5):
            if prob_list_read[g][-2]['top_logprobs'][i]['token']=='off':
                off_flg = True
                off_prob_list.append(np.exp(prob_list_read[g][-2]['top_logprobs'][i]['logprob']))
            elif prob_list_read[g][-2]['top_logprobs'][i]['token']=='on':
                on_flg = True
                on_prob_list.append(np.exp(prob_list_read[g][-2]['top_logprobs'][i]['logprob']))
            elif prob_list_read[g][-2]['top_logprobs'][i]['token']=='und':
                und_flg = True
                und_prob_list.append(np.exp(prob_list_read[g][-2]['top_logprobs'][i]['logprob']))
        if not off_flg:
            off_prob_list.append(0)
        if not on_flg:
            on_prob_list.append(0)
        if not und_flg:
            und_prob_list.append(0)
    return off_prob_list, on_prob_list, und_prob_list

# ============= Code for generate P(h) ===============
def extract_probs(gpt_fb, objects, top_n=5):
    token_logprob = {}
    for i in range(len(gpt_fb)):
        if gpt_fb[i]['token'] in objects:
            obj = gpt_fb[i]['token']
            obj_toplogs = gpt_fb[i+3]['top_logprobs']
            token_logprob[obj]={}
            for j in range(top_n):
                candi_token = obj_toplogs[j]['token']
                candi_prob = obj_toplogs[j]['logprob']
                token_logprob[obj][candi_token] = candi_prob #np.exp(candi_prob)
    return token_logprob

def cal_prob_of_h(h_star, token_logprob):
    obj_logprob = 0
    for obj in h_star.keys():
        status = h_star[obj]
        if status in token_logprob[obj].keys():
            tmp_logprob = token_logprob[obj][status]
        else:
            tmp_logprob = -10
        obj_logprob += tmp_logprob
    return obj_logprob, np.exp(obj_logprob)

def generate_all_statuses(N, M):
    def generate_status_helper(current_status):
        if len(current_status) == N:
            all_statuses.append(current_status.copy())
            return

        for state in range(M):
            current_status.append(STATES[state])
            generate_status_helper(current_status)
            current_status.pop()

    all_statuses = []
    generate_status_helper([])

    return all_statuses

# Example usage:
N = len(OBJECTS)  # Number of variables
M = len(STATES)  # Number of possible states for each variable

all_possible_statuses = generate_all_statuses(N, M)

In [ ]:
print('======= Before refine ========')
for i in range(GEN+1):
    if i==0:
        print('Rul*: ',end='')
    print(results_read['rules'][i])
    
print('======= After refine ========')
print('*%s'%results_read['rules'][0])
for i in range(GEN):
    print(' %s'%results_read['rules_refine'][i])

In [ ]:
off_prob_list, on_prob_list, und_prob_list = get_all_prob_lists(prob_list_read)
fig, ax = plt.subplots(1,3,figsize=(15,4))
x_axis=np.arange(0,GEN+1)
ax[0].plot(x_axis, [0.33]+off_prob_list,label='off')
ax[0].plot(x_axis, [0.33]+on_prob_list,label='on')
ax[0].plot(x_axis, [0.33]+und_prob_list,label='und')
ax[0].legend()
ax[0].set_title('P(Screen:?)')
uni_entropy, oht_entropy = 0, 0
for i in range(243):
    if i==3:
        prob = 1-242*1e-10
        oht_entropy += -prob*np.log(prob)
    uni_entropy += -(1/243)*np.log(1/243)
    oht_entropy += -1e-10*np.log(1e-10)

entropy_list = []
for i in range(GEN):
    entropy = cal_entropy(prob_list_read[i])
    entropy_list.append(entropy)
    
ax[1].plot(entropy_list,label='GPT return')
ax[1].plot([0,GEN-1],[uni_entropy,uni_entropy], label='Uniform')
ax[1].plot([0,GEN-1],[oht_entropy,oht_entropy], label='One-hot')
ax[1].legend(fontsize=16)
ax[1].set_title('Entropy')

d0_corr_list = eval_get_corr_d0(d0_str, results_read)
h_corr_list = eval_get_corr_h(results_read)
ax[2].plot(d0_corr_list, label='ht_corr_in_d0')
ax[2].plot(h_corr_list, label='dt_corr_in_h*')
ax[2].legend(fontsize=16)
ax[2].set_title('h/d count')

In [ ]:
from utils.evaluations import *

In [ ]:
def cal_screen_off_hbar(rule_list, h_bar):
    cnt_screen, cnt_h = 0, 0
    for r in rule_list:
        if r['screen']=='off':
            cnt_screen += 1
        if r==h_bar:
            cnt_h += 1
    return cnt_screen, cnt_h, len(rule_list)

def updata_stats(stats, results_read,):
    d0_str = results_read['d_sampled'][0]
    d0_corr_list = eval_get_corr_d0(d0_str, results_read)
    h_bar = results_read['rules'][0]
    h_bar['screen']='off'
    cnt_screen_start, cnt_h, all_cnt = cal_screen_off_hbar(results_read['rules_refine'][:0], h_bar)
    cnt_screen, cnt_h, all_cnt = cal_screen_off_hbar(results_read['rules_refine'][-AVG_BACK:], h_bar)
    cnt_corr_d0 = np.sum(d0_corr_list[-AVG_BACK:])
    all_cnt_corr_d0 = AVG_BACK*8
    stats["cnt_screen_start"].append(cnt_screen_start)
    stats["cnt_screen_end"].append(cnt_screen)
    stats["cnt_h"].append(cnt_h)
    stats["cnt_all"].append(all_cnt)
    stats["cnt_d0"].append(cnt_corr_d0)
    stats["cnt_d0_all"].append(all_cnt_corr_d0)

def regularize_results(results_read):    
    for i in range(len(results_read['rules'])):
        if type(results_read['rules'][i]) != dict:
            results_read['rules'][i] = json.loads(results_read['rules'][i].split("Rule: ")[1])

    for i in range(len(results_read['rules_refine'])):
        if type(results_read['rules_refine'][i]) != dict:
            results_read['rules_refine'][i] = json.loads(results_read['rules_refine'][i])  
    return results_read

def get_results_read(exp_path_load):
    #save_path = os.path.join(exp_path_load, 'prob_list_all.json') 
    save_path2 = os.path.join(exp_path_load, 'other_results_all.json')
    #prob_list_read = json.load( open( save_path ))
    results_read = json.load(open(save_path2))
    return results_read

In [ ]:
AVG_BACK = 1
GEN = 6
stats_none = {"cnt_screen_start":[],"cnt_screen_end":[],"cnt_h":[],"cnt_all":[],"cnt_d0":[],"cnt_d0_all":[]}
stats_self = {"cnt_screen_start":[],"cnt_screen_end":[],"cnt_h":[],"cnt_all":[],"cnt_d0":[],"cnt_d0_all":[]}
stats_hypo = {"cnt_screen_start":[],"cnt_screen_end":[],"cnt_h":[],"cnt_all":[],"cnt_d0":[],"cnt_d0_all":[]}

#allpath = os.walk("E://P5_iICL//iterated_learning_exp//exp_logs//add_interaction")
allpath = os.walk("E://P5_iICL//iterated_learning_exp//exp_logs_gpt-3.5-turbo-0125//add_interaction")
for path, dir_list, file_list in allpath:
    for dir_name in dir_list:
        results_read = get_results_read(os.path.join(path, dir_name))
        results_read = regularize_results(results_read)
        if dir_name.startswith('none'): 
            updata_stats(stats_none, results_read)
        elif dir_name.startswith('self'):
            updata_stats(stats_self, results_read)
        elif dir_name.startswith('hypothesis'):
            updata_stats(stats_hypo, results_read)

In [ ]:
OBS_KEY = 'cnt_d0'
print('======= %s ======'%OBS_KEY)
print(np.mean(stats_none[OBS_KEY]), np.var(stats_none[OBS_KEY]))
print(np.mean(stats_self[OBS_KEY]), np.var(stats_self[OBS_KEY]))
print(np.mean(stats_hypo[OBS_KEY]), np.var(stats_hypo[OBS_KEY]))

OBS_KEY = 'cnt_screen_start'
print('======= %s ======'%OBS_KEY)
print(np.mean(stats_none[OBS_KEY]), np.var(stats_none[OBS_KEY]))
print(np.mean(stats_self[OBS_KEY]), np.var(stats_self[OBS_KEY]))
print(np.mean(stats_hypo[OBS_KEY]), np.var(stats_hypo[OBS_KEY]))

OBS_KEY = 'cnt_screen_end'
print('======= %s ======'%OBS_KEY)
print(np.mean(stats_none[OBS_KEY]), np.var(stats_none[OBS_KEY]))
print(np.mean(stats_self[OBS_KEY]), np.var(stats_self[OBS_KEY]))
print(np.mean(stats_hypo[OBS_KEY]), np.var(stats_hypo[OBS_KEY]))

OBS_KEY = 'cnt_h'
print('======= %s ======'%OBS_KEY)
print(np.mean(stats_none[OBS_KEY]), np.var(stats_none[OBS_KEY]))
print(np.mean(stats_self[OBS_KEY]), np.var(stats_self[OBS_KEY]))
print(np.mean(stats_hypo[OBS_KEY]), np.var(stats_hypo[OBS_KEY]))

In [ ]:
# # ============= Code for generate P(h) ===============
# def extract_probs(gpt_fb, objects, top_n=5):
#     token_logprob = {}
#     for i in range(len(gpt_fb)):
#         if gpt_fb[i]['token'] in objects:
#             obj = gpt_fb[i]['token']
#             obj_toplogs = gpt_fb[i+3]['top_logprobs']
#             token_logprob[obj]={}
#             for j in range(top_n):
#                 candi_token = obj_toplogs[j]['token']
#                 candi_prob = obj_toplogs[j]['logprob']
#                 token_logprob[obj][candi_token] = candi_prob #np.exp(candi_prob)
#     return token_logprob

# def cal_prob_of_h(h_star, token_logprob):
#     obj_logprob = 0
#     for obj in h_star.keys():
#         status = h_star[obj]
#         if status in token_logprob[obj].keys():
#             tmp_logprob = token_logprob[obj][status]
#         else:
#             tmp_logprob = -10
#         obj_logprob += tmp_logprob
#     return obj_logprob, np.exp(obj_logprob)

# def draw_pic(hd_fb_probs, h_star, d0_str, ax=None, y_log=True, ylim=None, legend=True, x_tickle=True, xlabel=None, star=True, color_type='screen'):
#     h_bar = copy.deepcopy(h_star)
#     h_bar['screen']='off'
#     if ax is None:
#         fig, ax = plt.subplots(1,1,figsize=(15,5))
#     token_logprob = extract_probs(hd_fb_probs, OBJECTS, top_n=5)

#     prob_list, corr_list, screen_list = [], [], []
#     for i in range(len(all_possible_statuses)):
#         h_tmp = {}
#         for j in range(len(OBJECTS)):
#             h_tmp[OBJECTS[j]] = all_possible_statuses[i][j]
#         if h_tmp == h_star:
#             h_star_idx = i
#         if h_tmp == h_bar:
#             h_bar_idx = i
#         obj_logprob, obj_prob = cal_prob_of_h(h_tmp, token_logprob)
#         prob_list.append(obj_prob)

#         d0_pairs = dlist_to_pairs(d0_str.split('\n'))      
#         corr_cnt,_ = count_corr_d0_pairs(d0_pairs, h_tmp)
#         corr_list.append(corr_cnt)
#         if all_possible_statuses[i][-1]=='on':
#             screen_list.append(0)
#         elif all_possible_statuses[i][-1]=='off':
#             screen_list.append(1)
#         else:
#             screen_list.append(2)
#     prob_list = np.array(prob_list)
#     corr_list = np.array(corr_list)
#     screen_list = np.array(screen_list)
#     x_axis = np.arange(0,243)

#     if color_type=='d0':
#         ALPHA_LIST = [0.05, 0.2, 0.3, 0.45, 0.55, 0.7, 0.8, 0.9 , 1]#[0.05, 0.2, 0.3, 0.5, 1]
#         for i in range(len(ALPHA_LIST)):
#             mask = corr_list==i
#             if i>0:
#                 label = "%d corr"%i
#             else:
#                 label = None
#             ax.bar(x_axis[mask],prob_list[mask],width=1, color='royalblue',alpha=ALPHA_LIST[i],label=label)
#     elif color_type=='screen':
#         COLOR_LIST = ['#f8ac8c', 'royalblue','#9e9e9e']#'#2878b5',
#         LABELS = ['on', 'off', 'und']
#         for i in range(len(COLOR_LIST)):
#             mask = screen_list==i
#             ax.bar(x_axis[mask],prob_list[mask],width=1, color=COLOR_LIST[i],alpha=0.7,label=LABELS[i])
        
#     if y_log:
#         ax.set_yscale('log')
#     ax.set_xlim(-3, 245)
#     if ylim is not None:
#         ax.set_ylim(ylim)
#     if star:
#         ax.plot((h_star_idx), (prob_list[h_star_idx]), color='red',alpha=1,linestyle=' ',marker='*',markersize=10, label=r'$h^*$')
#         ax.plot((h_bar_idx), (prob_list[h_bar_idx]), color='red',alpha=1,linestyle=' ',marker='+',markersize=10, label=r'$\hat{h}$')
#     if legend:
#         ax.legend(fontsize=16, ncol=5)
#     if not x_tickle:
#         ax.set_xticks([])
#     if xlabel is not None:
#         ax.set_xlabel(xlabel,fontsize=16)
#     return prob_list, corr_list

In [ ]:
# save_path = 'E://P5_iICL//iterated_learning_exp//exp_logs//add_interaction//none_h006_alltorng_seed10086//prob_list_all.json'
# save_path2 = 'E://P5_iICL//iterated_learning_exp//exp_logs//add_interaction//none_h006_alltorng_seed10086//other_results_all.json'
# prob_list_read = json.load( open( save_path ))
# results_read = json.load(open(save_path2))

In [ ]:
# h_star = results_read['rules'][0]
# d0_str = results_read['d_sampled'][0]
# all_possible_statuses = generate_all_statuses(N=len(OBJECTS), M=len(STATES), packed=False) # Control the fashion of h-243
# fig, ax = plt.subplots(6,1,figsize=(15,35))
# for i in range(GEN):
#     if i>0:
#         legend=False
#     else:
#         legend=True
#     prob_list, corr_list = draw_pic(prob_list_read[i],h_star,d0_str, ax[i], True, ylim=[1e-15,1],legend=legend, color_type='screen')